# 🔗 Introduction to LangChain Expression Language (LCEL)

## What is LCEL?

**LangChain Expression Language (LCEL)** is a declarative way to compose chains in LangChain. It provides a simple, intuitive syntax using the **pipe operator (`|`)** to connect different components together.

### Key Benefits of LCEL:
- **🚀 Simplified Syntax**: Chain components using the intuitive `|` operator
- **⚡ First-class Streaming Support**: Get tokens as soon as they're available from the LLM
- **🔄 Async Support**: Same chain works for both sync and async operations
- **🔧 Optimized Parallel Execution**: Automatically runs independent steps in parallel
- **📝 Transparent Tracing**: Easy debugging with LangSmith integration

### How LCEL Works:
```
Input → Component1 | Component2 | Component3 → Output
```

Each component in the chain:
1. Receives input from the previous component
2. Processes it
3. Passes the output to the next component

---

## Learning Objectives:
By the end of this notebook, you will:
1. Understand the basic LCEL syntax
2. Create a simple LLM chain using LCEL
3. Execute the chain with custom inputs
4. See a practical example with SQL query generation

In [1]:
# ## Install Libraries
# !pip install -qq langchain==0.3.11
# !pip install -qq langchain-openai==0.2.12

In [2]:
# !pip install -qq langchain_community

## Setup Open AI API credentials

In [3]:
import os
from dotenv import load_dotenv
load_dotenv()

True

## Connect to the LLM

In [4]:
from langchain_openai import ChatOpenAI

chatgpt = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

## Create LCEL LLM Chain

In [8]:
from langchain_core.prompts import ChatPromptTemplate

# create a prompt template to accept user queries
prompt_txt = "Explain {topic} in 1 line"
prompt_template = ChatPromptTemplate.from_template(prompt_txt)

# the chain has been formatted for better readability
# you could also write this as llmchain = prompt_template | chatgpt
llmchain = (prompt_template
              |
           chatgpt)

## Run the LLM Chain

In [9]:
response = llmchain.invoke({'topic' : 'Generative AI'})
print(response.content)

Generative AI refers to algorithms that can create new content, such as text, images, or music, by learning patterns from existing data.


In [10]:
from langchain_openai import ChatOpenAI
from langchain.chains import create_sql_query_chain
from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///Chinook.db")
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
chain = create_sql_query_chain(llm, db)
response = chain.invoke({"question": "How many employees are there"})

In [11]:
response

'SELECT COUNT("EmployeeID") AS num_employees\nFROM "Employees"'